In [144]:
%%writefile tests/example_string_short.py
from pipedata import AutoNode as Node
from pipedata import TrackedFile, InputTrackedFile,frame_init
import os

import time

symin, symout, index= frame_init()
print "[indexFile]",_indexFile
numberFile = InputTrackedFile('tests-number.txt')
letterFile = InputTrackedFile('tests-letter.txt')

# _output_kw = Node.from_func
@Node.from_func({
    "OUT":TrackedFile("tests-out5.txt"),
#     "BAM":TrackedFile( "test.fastq.bam"  )
})
def out5(  self, numberFile, letterFile ):
    '''
    some doc
    '''
    number = numberFile().open('r').read().strip()
    letter = letterFile().open('r').read().strip()
    with self.output_kw['OUT'].open("w") as f:
        f.write( 5 * (number+letter)+'\n')
    return


@Node.from_func({
    "OUT":TrackedFile("tests-out10.txt"),
#     "BAM":TrackedFile( "test.fastq.bam"  )
})
def out10(  self, numberFile, letterFile ):
    '''
    some doc
    '''
    number = numberFile().open('r').read().strip()
    letter = letterFile().open('r').read().strip()
    with self.output_kw['OUT'].open("w") as f:
        f.write( 10 * (number+letter)+'\n')
    return

@Node.from_func({
    "OUT":TrackedFile("tests-out15.txt"),
#     "BAM":TrackedFile( "test.fastq.bam"  )
})
def out15(  self, numberFile, letterFile ):
    '''
    some doc
    '''
    number = numberFile().open('r').read().strip()
    letter = letterFile().open('r').read().strip()
    with self.output_kw['OUT'].open("w") as f:
        f.write( 15 * (number+letter)+'\n')
    return


@Node.from_func({
    'OUT':TrackedFile('tests-combined.txt')
})
def make_combined( self, out5, out10, out15, ):
    lines = []
    [ lines.extend(list(x()['OUT'].open('r'))) for x in [out5,
                                                         out10,out15
                                                        ]]
    with self['OUT'].open('w') as f:
        map(f.write,lines)
    return 1


if __name__ == '__main__':
    from pipedata import index_file_flush

    TrackedFile.VERBOSE=0
    print('START' + 20*"-")
    symout()
    print(symout().input_kw['make_combined']['OUT'].open('r').read())
    index_file_flush()
    print('END' + 20*"-")
    
    import dill
    dill.dumps(symout)
#     dill.dumps(make_combined)
#     print symout
    
#     make_bam()
#     make_csv()
#     print("[middle]",middle())


Overwriting tests/example_string_short.py


In [1]:
!touch tests/__init__.py

In [6]:
!python2 test.py

[FRAME_INIT] _symbolicInputNode, _symbolicOutputNode
[Dillable]
.
----------------------------------------------------------------------
Ran 1 test in 0.062s

OK


In [225]:
%%writefile test.py
import unittest
unittest.TestCase.assertRaises

import os
# from pymisca.tree import getPathStack
import path
# def getPathStack(x):
#     return path.Path(x[0])

import subprocess
import shutil
import sys

from pipe_run import pipe_run
import imp
import sys
from pipedata import IndexedDiffFileError,IndexedMissingFileError
# import os as pip/e
def _shell(cmd,shell=True):
    sys.stderr.write("[CMD]%s\n"%cmd)
    return subprocess.check_output(cmd,shell=shell)
class PipeRunner(object):
    def __init__(self, key='pipe'):
        self.key = key
    def __call__(self,*a,**kw):
        key = self.key
        pipe = imp.load_source( key, key +'.py')
        pipe_run(pipe)
        return pipe

class test(unittest.TestCase):
    def test_import(self):
        pass
#         print "[CURR]",os.getcwd()
        pipe = imp.load_source('pipe','tests/example_string_short.py')
        
#         import tests.example_string_short as pipe
#         pipe.__file__ = os.path.realpath(pipe.__file__)
        return pipe
        
    def test_dillable(self):
        pipe = self.test_import()
#         imp/reload(pipe)
        import dill
        dill.dumps(pipe,)
        print("[Dillable]")
        
    def make_copy(self, name =None):
        pipe = self.test_import()
        if name is None:
            name = 'test_build'
            
        subprocess.check_output("mkdir -p {name}/ && rm -fr {name}/*".format(**locals())
                                ,shell=1)
#         modpath = name+'/pipe.py'
        d = path.Path(name)

        fn = pipe.__file__.replace('.pyc','.py')
        shutil.copy2(fn,d/"pipe.py" )
#         shutil.copy2('tests/example_string_short.py'.replace('.pyc','.py'), modpath )
        import pipedata
        fn = pipedata.__file__.replace('.pyc','.py')
        shutil.copy2( fn, d/"pipedata.py")
        
        return name
        
    def test_1(self):
        dirname = self.make_copy()
        with path.Path(dirname) as d:
            print _shell('''
    echo "1"> tests-number.txt; 
    echo a>tests-letter.txt; 
            '''.format(**locals()))    
#             import pipe
#             pipe= imp.load_source( 'pipe', 'pipe.py')
#             pipe_run(pipe)
            pipe = PipeRunner()()
            print pipe._symbolicOutputNode().input_kw['make_combined']['OUT'].open('r').read()    
        return dirname
    
    def test_indexedDiffFile(self):
        with path.Path(self.test_1()).makedirs_p() as d:
            _shell('''
touch tests-out5.txt
''')
            self.assertRaises( IndexedDiffFileError, PipeRunner())
            pass
        return
    def test_indexedMissingFile(self):
        with path.Path(self.test_1()).makedirs_p() as d:

            _shell('''
rm tests-out5.txt             
''')
            self.assertRaises( IndexedMissingFileError, PipeRunner())
            pass
        return
if __name__ == '__main__':
#     runner = unittest.TextTestRunner()
#     runner.run(test())
    unittest.main()

Overwriting test.py


In [ ]:
# import 

In [55]:
import imp
# imp.load_module?

In [228]:
!python2 test.py

[FRAME_INIT] _symbolicInputNode, _symbolicOutputNode
[indexFile] tests/example_string_short.py.index
[CMD]
    echo "1"> tests-number.txt; 
    echo a>tests-letter.txt; 
            

[FRAME_INIT] _symbolicInputNode, _symbolicOutputNode
[indexFile] pipe.py.index
START--------------------
CHECKING_NODE:<Node with func:_symbolicOutputNode>
CHECKING_NODE:<Node with func:_symbolicInputNode>
[CHANGED_SAME]:<Node with func:_symbolicInputNode>
CHECKING_NODE:<Node with func:InputFileNode>
CHANGED_OUTPUT:<Node with func:InputFileNode>
CHECKING_NODE:<Node with func:out5>
CHANGED_OUTPUT:<Node with func:out5>
CHECKING_NODE:<Node with func:out10>
CHANGED_OUTPUT:<Node with func:out10>
CHECKING_NODE:<Node with func:out15>
CHANGED_OUTPUT:<Node with func:out15>
CHECKING_NODE:<Node with func:make_combined>
CHANGED_OUTPUT:<Node with func:make_combined>
CHANGED_UPSTREAM:<Node with func:_symbolicOutputNode>
RUNNING:<Node with func:_symbolicOutputNode>
RUNNING:<Node with func:InputFileNode>
RUNNING:<Node wi

In [62]:
import pymisca.header
# pymisca.header.f
import pymisca.ext as pyext
# pyext.file__asModule??
# pyext.AttoDirectory__toDirtt
from pymisca.atto_jobs import AttoModule
# ??AttoModule

In [1]:
import tests.example_string_short as pipe

from pipe_run import pipe_run
pipe_run(pipe)


[FRAME_INIT] _symbolicInputNode, _symbolicOutputNode
START--------------------
CHECKING_NODE:<Node with func:_symbolicOutputNode>
CHECKING_NODE:<Node with func:_symbolicInputNode>
[CHANGED_SAME]:<Node with func:_symbolicInputNode>
CHECKING_NODE:<Node with func:InputFileNode>
CHANGED_OUTPUT:<Node with func:InputFileNode>
CHECKING_NODE:<Node with func:out5>
CHANGED_OUTPUT:<Node with func:out5>
CHECKING_NODE:<Node with func:out10>
CHANGED_OUTPUT:<Node with func:out10>
CHECKING_NODE:<Node with func:out15>
CHANGED_OUTPUT:<Node with func:out15>
CHECKING_NODE:<Node with func:make_combined>
CHANGED_OUTPUT:<Node with func:make_combined>
CHANGED_UPSTREAM:<Node with func:_symbolicOutputNode>
RUNNING:<Node with func:_symbolicOutputNode>
RUNNING:<Node with func:InputFileNode>
RUNNING:<Node with func:out5>
RUNNING:<Node with func:out10>
RUNNING:<Node with func:out15>
RUNNING:<Node with func:make_combined>
END--------------------


In [2]:
# !ls -lhtr pipedata.py

In [3]:
%%bash 
set -x
exec 2>&1

mkdir -p tests
rm tests/*.txt

# 
ln pipedata.py tests -f 
rm -f tests/*.index
rm -f tests/test.index tests/out*.txt

echo "1"> tests/number.txt; 
echo a>tests/letter.txt; 
python2 tests/example_string_short.py

touch tests/out5.txt
python2 tests/example_string_short.py

rm tests/out5.txt  
### when missing a cached file 
## it needs to be detected and fixed
python2 tests/example_string_short.py

echo something> tests/out5.txt
echo "2">tests/number.txt; 
echo b>tests/letter.txt; 
python2 tests/example_string_short.py

+ mkdir -p tests
+ rm tests/combined.txt tests/letter.txt tests/number.txt tests/out10.txt tests/out15.txt tests/out5.txt
+ ln pipedata.py tests -f
+ rm -f tests/example_string_short.py.index
+ rm -f tests/test.index 'tests/out*.txt'
+ echo 1
+ echo a
+ python2 tests/example_string_short.py
[FRAME_INIT] _symbolicInputNode, _symbolicOutputNode
START--------------------
CHECKING_NODE:<Node with func:_symbolicOutputNode>
CHECKING_NODE:<Node with func:_symbolicInputNode>
[CHANGED_SAME]:<Node with func:_symbolicInputNode>
CHECKING_NODE:<Node with func:InputFileNode>
CHANGED_OUTPUT:<Node with func:InputFileNode>
CHECKING_NODE:<Node with func:out5>
CHANGED_OUTPUT:<Node with func:out5>
CHECKING_NODE:<Node with func:out10>
CHANGED_OUTPUT:<Node with func:out10>
CHECKING_NODE:<Node with func:out15>
CHANGED_OUTPUT:<Node with func:out15>
CHECKING_NODE:<Node with func:make_combined>
CHANGED_OUTPUT:<Node with func:make_combined>
CHANGED_UPSTREAM:<Node with func:_symbolicOutputNode>
RUNNING:<Node with

+ exec


In [ ]:
import unittest
unittest.TestCase.assertRaises

class test(unittest.TestCase):
    def runTest(self):
        a_cls.middle()
        a_cls.findA()
        self.assertRaises( AssertionError, a_cls.findB )
#         self.assertRaises(AssertionError, a_cls.findC )
if __name__ == '__main__':
    runner = unittest.TextTestRunner()
    runner.run(test())

In [4]:
%%bash 
set -x
exec 2>&1

mkdir -p tests
rm tests/*.txt

# 
ln pipedata.py tests -f 
rm -f tests/*.index
rm -f tests/test.index tests/out*.txt

echo "1"> tests/number.txt; 
echo a>tests/letter.txt; 
python2 tests/example_string_short.py

echo "2">tests/number.txt; 
echo b>tests/letter.txt; 
python2 tests/example_string_short.py

+ mkdir -p tests
+ rm tests/combined.txt tests/letter.txt tests/number.txt tests/out10.txt tests/out15.txt tests/out5.txt
+ ln pipedata.py tests -f
+ rm -f tests/example_string_short.py.index
+ rm -f tests/test.index 'tests/out*.txt'
+ echo 1
+ echo a
+ python2 tests/example_string_short.py
[FRAME_INIT] _symbolicInputNode, _symbolicOutputNode
START--------------------
CHECKING_NODE:<Node with func:_symbolicOutputNode>
CHECKING_NODE:<Node with func:_symbolicInputNode>
[CHANGED_SAME]:<Node with func:_symbolicInputNode>
CHECKING_NODE:<Node with func:InputFileNode>
CHANGED_OUTPUT:<Node with func:InputFileNode>
CHECKING_NODE:<Node with func:out5>
CHANGED_OUTPUT:<Node with func:out5>
CHECKING_NODE:<Node with func:out10>
CHANGED_OUTPUT:<Node with func:out10>
CHECKING_NODE:<Node with func:out15>
CHANGED_OUTPUT:<Node with func:out15>
CHECKING_NODE:<Node with func:make_combined>
CHANGED_OUTPUT:<Node with func:make_combined>
CHANGED_UPSTREAM:<Node with func:_symbolicOutputNode>
RUNNING:<Node with

+ exec


In [69]:
%%writefile test.py

# import pymisca.ext as pyext
import subprocess
def t1():
    test_expect = '+ mkdir -p tests\n+ ln pipedata.py tests -f\n+ rm -f tests/test.index tests/out10.txt tests/out15.txt tests/out5.txt\n+ echo 1\n+ echo a\n+ python2 tests/example_string_short.py\nSTART--------------------\nRUNNING:<Node with func:make_combined>\nRUNNING:<Node with func:out5>\nRUNNING:<Node with func:out10>\nRUNNING:<Node with func:out15>\n1a1a1a1a1a\n1a1a1a1a1a1a1a1a1a1a\n1a1a1a1a1a1a1a1a1a1a1a1a1a1a1a\n\nEND--------------------\n+ python2 tests/example_string_short.py\nSTART--------------------\nSKIPPING:<Node with func:make_combined>\n1a1a1a1a1a\n1a1a1a1a1a1a1a1a1a1a\n1a1a1a1a1a1a1a1a1a1a1a1a1a1a1a\n\nEND--------------------\n+ echo 2\n+ echo b\n+ python2 tests/example_string_short.py\nSTART--------------------\nRUNNING:<Node with func:make_combined>\nRUNNING:<Node with func:out5>\nRUNNING:<Node with func:out10>\nRUNNING:<Node with func:out15>\n2b2b2b2b2b\n2b2b2b2b2b2b2b2b2b2b\n2b2b2b2b2b2b2b2b2b2b2b2b2b2b2b\n\nEND--------------------'


    test_cmd  = '''
    set -x
    exec 2>&1

    mkdir -p tests
    ln pipedata.py tests -f 
    rm -f tests/test.index tests/out*.txt

    echo "1"> tests/number.txt; 
    echo a>tests/letter.txt; 
    python2 tests/example_string_short.py


    python2 tests/example_string_short.py

    echo "2">tests/number.txt; 
    echo b>tests/letter.txt; 
    python2 tests/example_string_short.py
    '''
    print repr(subprocess.check_output(test_cmd,shell=1).strip())
    assert subprocess.check_output(test_cmd,shell=1).strip()==test_expect.strip()
t1()


Overwriting test.py


In [67]:
%%bash
set -x
echo "1"> tests/number.txt; 
echo a>tests/letter.txt; 
python2 tests/example_string_short.py


python2 tests/example_string_short.py

echo "2">tests/number.txt; 
echo b>tests/letter.txt; 
python2 tests/example_string_short.py

[FRAME_INIT] _symbolicInputNode, _symbolicOutputNode
START--------------------
tests/number.txt [(1575036226.7871823, 1575036243.1912405), (2, 2)]
tests/letter.txt [(1575036226.7871823, 1575036243.1912405), (2, 2)]
tests/number.txt [(1575036226.7871823, 1575036243.1912405), (2, 2)]
[CheckingChange]:tests/number.txt.changed=1.state=INPUT_INDEX_QUEUED
tests/letter.txt [(1575036226.7871823, 1575036243.1912405), (2, 2)]
[CheckingChange]:tests/letter.txt.changed=1.state=INPUT_INDEX_QUEUED
tests/number.txt [(1575036226.7871823, 1575036243.1912405), (2, 2)]
[CheckingChange]:tests/number.txt.changed=1.state=INPUT_INDEX_QUEUED
tests/letter.txt [(1575036226.7871823, 1575036243.1912405), (2, 2)]
[CheckingChange]:tests/letter.txt.changed=1.state=INPUT_INDEX_QUEUED
tests/number.txt [(1575036226.7871823, 1575036243.1912405), (2, 2)]
[CheckingChange]:tests/number.txt.changed=1.state=INPUT_INDEX_QUEUED
tests/letter.txt [(1575036226.7871823, 1575036243.1912405), (2, 2)]
[CheckingChange]:tests/letter.tx

+ echo 1
+ echo a
+ python2 tests/example_string_short.py
+ python2 tests/example_string_short.py
+ echo 2
+ echo b
+ python2 tests/example_string_short.py


In [37]:
!python2 test.py

+ exec
+ mkdir -p tests
+ ln pipedata.py tests -f
+ rm -f tests/test.index tests/out*.txt
+ echo 1
+ echo a
+ python2 tests/example_string_short.py
[ADDED] _symbolicInputNode, _symbolicOutputNode
START--------------------
END--------------------
<Node with func:<lambda>>
+ python2 tests/example_string_short.py
[ADDED] _symbolicInputNode, _symbolicOutputNode
START--------------------
END--------------------
<Node with func:<lambda>>
+ echo 2
+ echo b
+ python2 tests/example_string_short.py
[ADDED] _symbolicInputNode, _symbolicOutputNode
START--------------------
END--------------------
<Node with func:<lambda>>
+ exec
Traceback (most recent call last):
  File "test.py", line 30, in <module>
    t1()
  File "test.py", line 28, in t1
    assert subprocess.check_output(test_cmd,shell=1).strip()==test_expect.strip()
AssertionError
